# 03 — Fine-tuning NLLB (Cebuano → Tagalog)

**Purpose:**
 Fine-tune the NLLB model on your aligned Cebuano–Tagalog dataset.

**Key stages:**

1. **Load TSVs** (train/dev).
2. **Tokenize** both sides, adding language tags (`<<ceb_Latn>>`, forced `tgl_Latn` BOS).
3. **Train** with `Seq2SeqTrainer` (2 epochs, 8-sentence batches).
4. **Evaluate** automatically at each epoch.
5. **Save** model and metrics.

**Outputs:**

- `experiments/finetune/` folder containing:
  - Trained model weights.
  - Tokenizer config.
  - `metrics.json` (evaluation scores).

### Fine-tuning the NLLB model

This cell runs the `finetune.py` script to train a multilingual NLLB translation model (`facebook/nllb-200-distilled-600M`) on the prepared Cebuano–Tagalog parallel dataset.

In [1]:
!python ../src/train/finetune.py \
  --train ../data/processed/train.tsv \
  --dev ../data/processed/dev.tsv \
  --out ../experiments/finetune

Resolved codes → src: 'ceb_Latn' as 'ceb_Latn' (id=256035) | tgt: 'tgl_Latn' as 'tgl_Latn' (id=256174)
Train samples: 22,851 | Dev samples: 2,930
{'loss': 2.4677, 'grad_norm': 3.28125, 'learning_rate': 1.982849142457123e-05, 'epoch': 0.02}
{'loss': 2.3679, 'grad_norm': 3.34375, 'learning_rate': 1.9653482674133707e-05, 'epoch': 0.04}
{'loss': 2.3481, 'grad_norm': 3.53125, 'learning_rate': 1.9478473923696188e-05, 'epoch': 0.05}
{'loss': 2.3069, 'grad_norm': 3.328125, 'learning_rate': 1.9303465173258665e-05, 'epoch': 0.07}
{'loss': 2.2973, 'grad_norm': 3.0, 'learning_rate': 1.9128456422821142e-05, 'epoch': 0.09}
{'loss': 2.1498, 'grad_norm': 2.703125, 'learning_rate': 1.895344767238362e-05, 'epoch': 0.11}
{'loss': 2.1989, 'grad_norm': 3.375, 'learning_rate': 1.87784389219461e-05, 'epoch': 0.12}
{'loss': 2.2096, 'grad_norm': 3.5625, 'learning_rate': 1.8603430171508577e-05, 'epoch': 0.14}
{'loss': 2.1677, 'grad_norm': 2.859375, 'learning_rate': 1.8428421421071054e-05, 'epoch': 0.16}
{'loss'

`torch_dtype` is deprecated! Use `dtype` instead!

Tokenizing train: 100%|██████████| 22851/22851 [00:15<00:00, 1470.53 examples/s]

Tokenizing dev: 100%|██████████| 2930/2930 [00:01<00:00, 1810.79 examples/s]
d:\OneDrive\Documents\My Learning Resource\University Courses\DLSU\2025-26\T1\CSC715M\assignments\mc02\src\train\finetune.py:239: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(

  1%|          | 50/5714 [00:11<18:54,  4.99it/s]
                                                 

  2%|▏         | 100/5714 [00:22<20:13,  4.63it/s]
                                                  

  3%|▎         | 150/5714 [00:33<21:53,  4.24it/s]
                                                  

  4%|▎         | 200/5714 [00:42<17:46,  5.17it/s]
                                                  

  4%|▍         | 250/5714 [01:01<45:47,  1.99it/s]
                             

The script performs the following steps:

* Loads the training and development splits (`train.tsv` and `dev.tsv`) from `../data/processed/`.
* Initializes the NLLB tokenizer and model, automatically resolving the correct language codes (`ceb_Latn` → `tgl_Latn`).
* Preprocesses each sentence pair, prefixing Cebuano inputs with the source language tag for proper conditioning.
* Fine-tunes the model for 2 epochs using mixed precision (FP16 or BF16 if available on GPU).
* Saves the trained model, checkpoints, and training configuration in `../experiments/finetune/`.

After execution, you should see logs showing tokenization progress, GPU precision mode, and epoch-by-epoch training metrics. The final trained model will be stored in the specified `finetune` experiment folder.

### Evaluate the fine-tuned model on the test set

This cell loads the **fine-tuned NLLB model** from `../experiments/finetune/`, translates the test set, and computes **BLEU** and **chrF2** with SacreBLEU.

In [2]:
!python ../src/eval/evaluate.py \
  --model_dir ../experiments/finetune \
  --test_tsv ../data/processed/test.tsv \
  --out_json ../experiments/finetune/metrics.json \
  --save_hyp ../experiments/finetune/hyp.txt \
  --code_src ceb_Latn --code_tgt tgl_Latn


{
  "BLEU": 29.83,
  "chrF2": 56.32,
  "ref_len": 85119,
  "sys_len": 89467,
  "sacrebleu_version": "2.5.1",
  "n_samples": 2750,
  "model_dir": "../experiments/finetune",
  "codes": {
    "src": "ceb_Latn",
    "tgt": "tgl_Latn"
  },
  "decoding": {
    "beams": 5,
    "max_new_tokens": 200,
    "batch_size": 16
  }
}


`torch_dtype` is deprecated! Use `dtype` instead!

Translating:   0%|          | 0/172 [00:00<?, ?it/s]C:\Users\julia\AppData\Roaming\Python\Python312\site-packages\transformers\generation\utils.py:1733: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed in v5. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(

Translating: 100%|██████████| 172/172 [11:13<00:00,  3.92s/it]


What happens

* Loads `test.tsv` (columns: `src`, `tgt`) and the tokenizer/model from `../experiments/finetune`.
* Resolves NLLB language tags and **forces BOS** to Tagalog (`tgl_Latn`) for decoding consistency.
* Translates the source sentences in **mini-batches** (default batch size = 16) using beam search (default beams = 5).
* Writes the system outputs to `../experiments/finetune/hyp.txt`.
* Scores the outputs against references with SacreBLEU and saves: